In [5]:
## INIT
# This is a bash notebook to structure NHP Freesurfer processing
# c.klink@nin.knaw.nl

fsSUB=Danny_enhanced
# the 'enhanced' volume has the WM artificially brightened after processing with NMT template

echo 'Working on '$fsSUB

Working on Danny_enhanced


In [6]:
BRAIN_0=$fsSUB.nii.gz
BRAIN_0_path=/Users/chris/Dropbox/CURRENT_PROJECTS/NHP_MRI/T1/$fsSUB  
NMT_path=/NHP_MRI/Template/NMT
tmpdir=~/fs_tmp
SCRIPT_path=$(pwd)

In [7]:
## STEP 00 - Prepping the data & initiate #################################
# copy to temp folder
mkdir -p $tmpdir
cp $BRAIN_0_path/$BRAIN_0 $tmpdir/tmp_brain.nii.gz
# align rigidly to NMT
flirt -in $tmpdir/tmp_brain.nii.gz \
    -ref $NMT_path/NMT.nii.gz \
    -out $tmpdir/tal_brain.nii.gz \
    -omat $tmpdir/brain_to_tal.mat \
    -dof 6

In [ ]:
# resample to 0.5 mm iso    
mri_convert -i $tmpdir/tal_brain.nii.gz \
    -o $tmpdir/tal_brain_rs.nii.gz \
    -vs 0.5 0.5 0.5
# fake the header to make fs think we have 1 mm voxels   
3drefit -xdel 1.0 -ydel 1.0 -zdel 1.0 -keepcen $tmpdir/tal_brain_rs.nii.gz

In [11]:
# additional allignment to make things work
# 1 - center between left/right should be at 128
# 2 - AC and PC should be at same height
freeview -v $tmpdir/tal_brain_rs.nii.gz
# save the corrected volume as >> tal_brain_rs2.nii.gz 
# and find the center of the volume

MRItoBSpline degree 3
MRItoBSpline degree 3
/Users/chris/fs_tmp/tal_brain_rs2.nii.gz saved successfully.


In [12]:
## STEP 01 - CROP THE VOLUME TO 25x256x256 ###############################
CTR=(63 88 56)  # (x y z)

mri_convert -i $tmpdir/tal_brain_rs2.nii.gz \
    -o $tmpdir/orig.mgz \
    --crop ${CTR[0]} ${CTR[1]} ${CTR[2]} \
    --conform -nc

mri_convert.bin -i /Users/chris/fs_tmp/tal_brain_rs2.nii.gz -o /Users/chris/fs_tmp/orig.mgz --crop 63 88 56 --conform -nc 
$Id: mri_convert.c,v 1.226 2016/02/26 16:15:24 mreuter Exp $
reading from /Users/chris/fs_tmp/tal_brain_rs2.nii.gz...
TR=0.00, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (1, -0, 0)
j_ras = (-0, 1, 0)
k_ras = (-0, -0, 1)
Reslicing using trilinear interpolation 
writing to /Users/chris/fs_tmp/orig.mgz...


In [13]:
## STEP 02 - SKULL-STRIP ##################################################
WST=25 # play with this parameter to get better skullstrip (30/24/15)
recon-all -i $tmpdir/orig.mgz -subjid $fsSUB \
    -autorecon1 -no-wsgcaatlas -wsthresh $WST \
    -notal-check -notalairach

# check skullstripping result with:
freeview -v $SUBJECTS_DIR/$fsSUB/mri/T1.mgz \
    $SUBJECTS_DIR/$fsSUB/mri/brainmask.mgz

Subject Stamp: freesurfer-Darwin-OSX-stable-pub-v6.0.0-2beb96c
Current Stamp: freesurfer-Darwin-OSX-stable-pub-v6.0.0-2beb96c
INFO: SUBJECTS_DIR is /Users/chris/tutorial_data/buckner_data/tutorial_subjs
Actual FREESURFER_HOME /Users/chris/Documents/MRI/freesurfer
Darwin ChrisKlink-rMBP.local 18.2.0 Darwin Kernel Version 18.2.0: Mon Nov 12 20:24:46 PST 2018; root:xnu-4903.231.4~2/RELEASE_X86_64 x86_64
/Users/chris/tutorial_data/buckner_data/tutorial_subjs/Danny_enhanced
\n mri_convert /Users/chris/fs_tmp/orig.mgz /Users/chris/tutorial_data/buckner_data/tutorial_subjs/Danny_enhanced/mri/orig/001.mgz \n
mri_convert.bin /Users/chris/fs_tmp/orig.mgz /Users/chris/tutorial_data/buckner_data/tutorial_subjs/Danny_enhanced/mri/orig/001.mgz 
$Id: mri_convert.c,v 1.226 2016/02/26 16:15:24 mreuter Exp $
reading from /Users/chris/fs_tmp/orig.mgz...
TR=0.00, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, 0, 0)
j_ras = (0, 0, -1)
k_ras = (0, 1, 0)
writing to /Users/chris/tutorial_data/buckner_data/tut

Processing:.................................................................Done
Processing:.................................................................Done
Processing:.................................................................Done
Processing:.................................................................Done
Processing:.................................................................Done
Processing:.................................................................Done
Processing:.................................................................Done
Processing:.................................................................Done
Processing:.................................................................Done
Processing:.................................................................Done
Processing:.................................................................Done
Processing:.................................................................Done
Processing:.................

      ambiguous basin, merged: at least 82 ambiguous voxels; size: 540735 voxels
      ambiguous basin, merged: at least 8 ambiguous voxels; size: 419 voxels
      ambiguous basin, merged: at least 20 ambiguous voxels; size: 7428 voxels
      ambiguous basin, merged: at least 27 ambiguous voxels; size: 17971 voxels
      ambiguous basin, merged: at least 27 ambiguous voxels; size: 19264 voxels
      ambiguous basin, merged: at least 23 ambiguous voxels; size: 11743 voxels
      ***** 11 basin(s) merged in 4 iteration(s)
      ***** 1013888 voxel(s) added to the main basin
      ambiguous basin, non merged: 11 ambiguous voxels; size: 29875 voxels
      ambiguous basin, non merged: 4 ambiguous voxels; size: 8390 voxels
done.

****************TEMPLATE DEFORMATION****************

      second estimation of the COG coord: x=128,y=128, z=120, r=9662 iterations
^^^^^^^^ couldn't find WM with original limits - expanding ^^^^^^

   GLOBAL      CSF_MIN=1, CSF_intensity=2, CSF_MAX=3 , nb = 43227

In [ ]:
# if necessary (cerebellum should be present!), rerun skullstrip with:
WST2=10
recon-all -subjid $fsSUB -skullstrip \
    -no-wsgcaatlas -wsthresh $WST2 \
    -notal-check -notalairach -clean-bm

# manually adjust brainmask.nii.gz untill ok
freeview -v $SUBJECTS_DIR/$fsSUB/mri/T1.mgz \
    $SUBJECTS_DIR/$fsSUB/mri/brainmask.mgz

In [ ]:
## STEP 03 - SEGMENTATION #################################################
# get corpus callosum and pons coordinates

CC[1]=129 #130 X
CC[2]=119 #116 Y
CC[3]=138 #140 Z
PONS[1]=127
PONS[2]=151
PONS[3]=119

# first stab at freesurfer reconstruction
recon-all -s $fsSUB -autorecon2-inflate1 -noaseg \
    -cc-crs ${CC[1]} ${CC[2]} ${CC[3]} \
    -pons-crs ${PONS[1]} ${PONS[2]} ${PONS[3]} \
    -notal-check -notalairach


# add control points to improve (see freesurfer documentation)
freeview -v $SUBJECTS_DIR/$fsSUB/mri/brainmask.mgz \
    $SUBJECTS_DIR/$fsSUB/mri/wm.mgz:colormap=heat:opacity=0.4 \
    -f $SUBJECTS_DIR/$fsSUB/surf/lh.orig.nofix:edgecolor=yellow \
    $SUBJECTS_DIR/$fsSUB/surf/rh.orig.nofix:edgecolor=yellow \
    $SUBJECTS_DIR/$fsSUB/surf/rh.inflated.nofix:visible=0 \
    $SUBJECTS_DIR/$fsSUB/surf/lh.inflated.nofix:visible=0

# pick seed point in hemispheres to try to fix the cutting error
LH[1]=144
LH[2]=11
LH[3]=129 # X Y Z left hemisphere
RH[1]=110
RH[2]=113
RH[3]=131 # X Y Z right hemisphere

# rerun taking control points into account
recon-all -s $fsSUB -autorecon2-inflate1 -noaseg \
    -cc-crs ${CC[1]} ${CC[2]} ${CC[3]} \
    -pons-crs ${PONS[1]} ${PONS[2]} ${PONS[3]} \
    -notal-check -notalairach

# repeat untill ok
# manually edit WM

# check in surface rendering using
tksurfer $MONKEY rh inflated.nofix

In [ ]:
## STEP 04 - INFLATE ######################################################
# this one will take many hours so make sure 
# you're happy with the previous results

recon-all -s $fsSUB -autorecon2-wm -noaseg \
    -cc-crs ${CC[1]} ${CC[2]} ${CC[3]} \
    -pons-crs ${PONS[1]} ${PONS[2]} ${PONS[3]}